# Let's clean the data buddy !

With that general overview out of the way, let's start cleaning the Airbnb data. 

In relation to the datasets provided for the Airbnb Kaggle competition, we will focus our cleaning efforts on two files:

* **train_users_2.csv** 
* **test_users.csv** 

For now, *sessions.csv* will be left aside.

## 1. Loading the Python libraries


In [1]:
import os

import numpy as np
import pandas as pd

## 2. Loading in the Data

The first step is to load the data from the CSV files using Python. 
To do this we will use the Pandas library and load the data from two files **train_users_2.csv** and **test_users.csv**. 

After loading, we will combine them into one dataset so that any cleaning (and later any other changes) will be done to all the data at once.

In [2]:
df_train = pd.read_csv("../input/train_users_2.csv")
df_train.sample(n=5) # Only display a few lines and not the whole dataframe

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
37470,ylve94xo1t,2012-08-21,20120821192258,2012-08-25,FEMALE,67.0,basic,0,en,other,craigslist,linked,Web,Windows Desktop,Chrome,US
126021,kkzha4d1cv,2013-11-19,20131119224018,2014-02-27,-unknown-,25.0,basic,0,en,direct,direct,linked,Web,Mac Desktop,Safari,US
201267,wtclsr7jqt,2014-06-08,20140608041738,NaN,-unknown-,NaN,basic,0,en,sem-brand,google,linked,Web,Mac Desktop,Safari,NDF
112765,37c24ao22s,2013-09-30,20130930173925,2013-10-23,FEMALE,34.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,US
136307,9bwep1bwcr,2013-12-29,20131229192951,NaN,-unknown-,NaN,basic,12,en,api,other,untracked,Android,Other/Unknown,-unknown-,NDF


In [3]:
df_test = pd.read_csv("../input/test_users.csv")
df_test.sample(n=5) # Only display a few lines and not the whole dataframe

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
11440,sixyqxiknd,2014-07-19,20140719161159,NaN,-unknown-,37.0,basic,0,en,direct,direct,linked,Moweb,Android Phone,Chrome Mobile
22267,0uksl1a731,2014-08-01,20140801203000,NaN,-unknown-,NaN,basic,25,en,direct,direct,untracked,iOS,iPhone,-unknown-
13884,n919s6ij41,2014-07-22,20140722142517,NaN,-unknown-,NaN,basic,0,en,sem-brand,google,omg,Web,Windows Desktop,Chrome
28323,r1i30av92f,2014-08-10,20140810132856,NaN,-unknown-,NaN,basic,25,en,sem-brand,bing,omg,iOS,iPhone,Mobile Safari
44935,2itqu9b8wd,2014-09-03,20140903131202,NaN,-unknown-,19.0,facebook,0,en,seo,google,linked,Web,Mac Desktop,Safari


In [4]:
# Combine into one dataset
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
df_all.head(n=5) # Only display a few lines and not the whole dataframe

,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,date_first_booking,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,signup_flow,signup_method,timestamp_first_active
0,direct,direct,NaN,NDF,2010-06-28,NaN,untracked,Chrome,Mac Desktop,-unknown-,gxn3p5htnn,en,Web,0,facebook,20090319043255
1,seo,google,38.0,NDF,2011-05-25,NaN,untracked,Chrome,Mac Desktop,MALE,820tgsjxq7,en,Web,0,facebook,20090523174809
2,direct,direct,56.0,US,2010-09-28,2010-08-02,untracked,IE,Windows Desktop,FEMALE,4ft3gnwmtx,en,Web,3,basic,20090609231247
3,direct,direct,42.0,other,2011-12-05,2012-09-08,untracked,Firefox,Mac Desktop,FEMALE,bjjt8pjhuk,en,Web,0,facebook,20091031060129
4,direct,direct,41.0,US,2010-09-14,2010-02-18,untracked,Chrome,Mac Desktop,-unknown-,87mebub9p4,en,Web,0,basic,20091208061105


## 3. Cleaning the timestamp

Once the data has been loaded and combined, the first cleaning step we will undertake is fixing the format of the dates.

Fixing date format is essential in order to **simplify** date manipulation.

In [5]:
df_all['date_account_created'] = pd.to_datetime(df_all['date_account_created'], format='%Y-%m-%d')
df_all['timestamp_first_active'] = pd.to_datetime(df_all['timestamp_first_active'], format='%Y%m%d%H%M%S')

df_all.head(n=5) # Only display a few lines and not the whole dataframe

,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,date_first_booking,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,signup_flow,signup_method,timestamp_first_active
0,direct,direct,NaN,NDF,2010-06-28,NaN,untracked,Chrome,Mac Desktop,-unknown-,gxn3p5htnn,en,Web,0,facebook,2009-03-19 04:32:55
1,seo,google,38.0,NDF,2011-05-25,NaN,untracked,Chrome,Mac Desktop,MALE,820tgsjxq7,en,Web,0,facebook,2009-05-23 17:48:09
2,direct,direct,56.0,US,2010-09-28,2010-08-02,untracked,IE,Windows Desktop,FEMALE,4ft3gnwmtx,en,Web,3,basic,2009-06-09 23:12:47
3,direct,direct,42.0,other,2011-12-05,2012-09-08,untracked,Firefox,Mac Desktop,FEMALE,bjjt8pjhuk,en,Web,0,facebook,2009-10-31 06:01:29
4,direct,direct,41.0,US,2010-09-14,2010-02-18,untracked,Chrome,Mac Desktop,-unknown-,87mebub9p4,en,Web,0,basic,2009-12-08 06:11:05


## 4. Remove booking date field

For every test data the column "date_first_booking" is missing, so we can't rely on this one for our prediction model.

**=> We must delete the column**

In [6]:
# Remove date_first_booking column
df_all.drop('date_first_booking', axis=1, inplace=True)

## 5. Clean the Age column

There are several age values that are clearly incorrect (unreasonably high or too low). 

In this step, we replace these incorrect values with "NaN", which literally stands for *Not a Number*, but implies we do not know the age value. In other words we are changing the incorrect values into missing values. 

In [7]:
# Remove outliers function
def remove_outlier(x, min_val=15, max_val=90):
    if np.isnan(x):
        return np.nan
    elif np.logical_or(x<=min_val, x>=max_val):
        return np.nan
    else:
        return x

In [8]:
# Fixing age column
df_all['age'] = df_all['age'].apply(lambda x: remove_outlier(x, min_val=15, max_val=90))
df_all['age'].fillna(-1, inplace=True)

df_all.sample(n=5)

,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,signup_flow,signup_method,timestamp_first_active
61451,direct,direct,-1.0,NDF,2013-02-20,untracked,IE,Windows Desktop,-unknown-,0qgygvdho7,en,Web,0,basic,2013-02-20 23:36:37
272292,seo,facebook,31.0,NaN,2014-09-25,untracked,Chrome,Mac Desktop,-unknown-,zp0tlljixt,en,Web,0,facebook,2014-09-25 15:58:05
131709,sem-non-brand,vast,45.0,NDF,2013-12-11,omg,Firefox,Mac Desktop,MALE,ii6fw0cvxm,en,Web,0,basic,2013-12-11 17:58:35
50874,direct,direct,40.0,NDF,2012-12-03,untracked,Safari,Mac Desktop,FEMALE,lpbqzt8xej,en,Web,0,facebook,2012-12-03 18:28:57
87012,direct,direct,42.0,NDF,2013-06-27,untracked,Safari,Mac Desktop,MALE,5y2wcicboc,en,Web,0,facebook,2013-06-27 22:37:56


As mentioned earlier, there are several more complicated ways to fill in the missing values in the age column. 

We are selecting this simple method for two main reasons:

* **Clarity**: this workshop is going to be long enough without adding the complication of a complex methodology for imputing missing ages.

* **Questionable results**: in my testing during the actual competition, I did test several more complex imputation methodologies. However, none of the methods I tested actually produced a better end result than the methodology outlined above.


## 6. Converting Age Column to Integers

Values inside the age columns are **floats**, it is better to have them as **integers**

In [9]:
df_all.age = df_all.age.astype(int)
df_all.sample(n=5)

,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,signup_flow,signup_method,timestamp_first_active
4158,other,craigslist,42,US,2011-04-11,untracked,IE,Windows Desktop,-unknown-,yxd68yu2u8,en,Web,3,basic,2011-04-11 02:58:47
15851,direct,other,61,NDF,2012-01-25,NaN,-unknown-,Other/Unknown,MALE,bpf7790z6b,en,iOS,6,basic,2012-01-25 19:36:08
124436,direct,direct,-1,NDF,2013-11-14,untracked,Firefox,Mac Desktop,FEMALE,4ykwrxwz6m,en,Web,0,basic,2013-11-14 06:24:41
6169,direct,direct,31,US,2011-07-02,linked,Chrome,Windows Desktop,FEMALE,1gqpf2tjgl,en,Web,0,basic,2011-07-02 23:59:31
136990,direct,direct,-1,NDF,2013-12-31,linked,Firefox,Mac Desktop,-unknown-,svabta5rsk,en,Web,0,basic,2013-12-31 22:05:33


## 7. Identify and fill additional columns with missing values

In [10]:
def check_NaN_Values_in_df(df):
    # searching for NaN values is all the columns
    for col in df:
        nan_count = df[col].isnull().sum()

        if nan_count != 0:
            print (col + " => "+  str(nan_count) + " NaN Values")
            
check_NaN_Values_in_df(df_all) 

country_destination => 62096 NaN Values
first_affiliate_tracked => 6085 NaN Values


It is **absolutely normal** that <i>country_destination</i> contains NaN values, it is the output field. NaN Values are test data.

However, **we need to fix** the column <i>first_affiliate_tracked</i> 

In [11]:
df_all['first_affiliate_tracked'].fillna(-1, inplace=True)

# We check there is no more NaN except in country_destination column
check_NaN_Values_in_df(df_all) 

df_all.sample(n=5)

country_destination => 62096 NaN Values


,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,signup_flow,signup_method,timestamp_first_active
167963,content,google,23,NDF,2014-03-26,omg,Firefox,Windows Desktop,FEMALE,fg99hiqubd,id,Web,0,facebook,2014-03-26 10:35:05
229916,direct,direct,-1,NaN,2014-07-25,untracked,-unknown-,Android Tablet,-unknown-,qvj260bybw,en,Android,12,google,2014-07-25 00:29:31
251612,sem-brand,google,57,NaN,2014-08-24,omg,Chrome,Mac Desktop,MALE,ipay3s7r6j,en,Web,0,facebook,2014-08-24 22:07:18
249353,direct,direct,-1,NaN,2014-08-21,untracked,-unknown-,Android Phone,-unknown-,wqm3v38ymx,en,Android,12,google,2014-08-21 06:05:54
65480,api,other,24,NDF,2013-03-15,untracked,-unknown-,iPhone,MALE,0g0o9uytfe,en,iOS,12,facebook,2013-03-15 20:02:27


## 8. Removing Users which registred before Feb. 2013

In [12]:
df_all = df_all[df_all['date_account_created'] > '2013-02-01']
df_all.sample(n=5)

,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,signup_flow,signup_method,timestamp_first_active
241969,direct,direct,38,NaN,2014-08-10,untracked,-unknown-,iPhone,MALE,hcukp47ncj,en,iOS,25,facebook,2014-08-10 20:43:23
155206,direct,direct,-1,NDF,2014-02-21,untracked,Safari,Mac Desktop,-unknown-,0cc4s01n3x,da,Web,0,basic,2014-02-21 01:51:34
156411,other,other,29,other,2014-02-24,untracked,Chrome,Mac Desktop,MALE,ig8belycxo,en,Web,0,basic,2014-02-24 20:41:00
164477,sem-brand,google,42,NDF,2014-03-18,omg,Safari,Mac Desktop,FEMALE,eintv31wbm,en,Web,0,basic,2014-03-18 01:20:58
108837,direct,direct,29,FR,2013-09-18,untracked,Chrome,Mac Desktop,-unknown-,7dnz95xnvi,en,Web,0,basic,2013-09-18 21:35:28


## 9. Saving the DataFrame to csv

In [13]:
# We create the output directory if necessary
if not os.path.exists("output"):
    os.makedirs("output")
    
# We export to csv
df_all.to_csv("output/cleaned.csv", sep=',', index=False)